# Chapter 01. 선형대수와 텐서 (Linear Algebra)
AI의 근간이 되는 수치 연산의 원리를 이해하고, 하드웨어 효율성을 고려한 텐서 조작법을 마스터합니다.

### 1. 텐서 구조와 차원(Rank) (Step 1)
- **목표**: 데이터의 차원 개념 정립 및 Shape 에러 방지.
- **Deep Dive**: Scalar(0D)부터 고차원 Tensor(3D+)까지의 계층 구조와 신경망 데이터 흐름 이해.
- **Engineering**: `ndim`과 `shape`를 활용한 런타임 데이터 규격 검증 습관화.

### 2. 행렬 연산 직접 구현 및 복잡도 분석 (Step 2)
- **목표**: 라이브러리 내부에서 일어나는 연산 과정을 코드로 이해.
- **Deep Dive**: 행렬 곱셈의 수학적 조건(앞 열 = 뒤 행) 및 3중 for문 구현을 통한 $O(n^3)$ 연산량 체감.
- **Engineering**: 대규모 연산 시 Python 표준 리스트 사용 지양 및 NumPy 활용 이유 증명.

### 3. 메모리 레이아웃과 캐시 최적화 (Step 3)
- **목표**: 데이터가 RAM에 물리적으로 저장되는 방식을 이해하여 연산 속도 최적화.
- **Deep Dive**: C-style(Row-major) vs F-style(Column-major)의 메모리 주소 배치 차이 분석.
- **Engineering**: CPU 캐시 적중률(Cache Hit)을 높이기 위한 연속적 메모리 접근(Spatial Locality) 원리 학습.

### 4. 브로드캐스팅 및 차원 조작 (Step 4)
- **목표**: 서로 다른 모양의 데이터를 효율적으로 연산하고 변형하는 기술 습득.
- **Deep Dive**: 브로드캐스팅의 가상 확장 원리와 성립 조건(차원 일치 및 1의 존재) 분석.
- **Engineering**: `Reshape`, `Squeeze`, `Newaxis`를 활용한 메모리 복사 없는(Zero-copy) 차원 변형 기법.

### 5. 선형 변환과 고유값 분해 기초 (Step 5)
- **목표**: 데이터의 특징을 추출하는 선형대수의 핵심 도구 이해.
- **Deep Dive**: 고유값(Eigenvalue)과 고유벡터(Eigenvector)의 기하학적 의미(변하지 않는 축).
- **Engineering**: 고유값 분해를 활용한 데이터 압축(PCA) 및 정보 손실 최소화 전략의 기초 마련.

# Chapter 01. 선형대수와 텐서 (Linear Algebra)

인공지능의 모든 데이터는 숫자 덩어리인 **텐서(Tensor)**로 표현됩니다. 
단순히 라이브러리를 사용하는 것을 넘어, 메모리 상에서 데이터가 어떻게 배치되고 연산되는지 **Deep Dive** 합니다.

### 1. 텐서의 계층적 구조와 차원(Rank)
- **Scalar (0D)**: 크기만 있는 단일 값. (예: Loss 값, Learning Rate)
- **Vector (1D)**: 숫자의 나열. 방향과 크기를 가짐. (예: 특징 벡터, 임베딩)
- **Matrix (2D)**: 행(Row)과 열(Column)로 구성된 표. (예: 데이터프레임, 흑백 이미지)
- **Tensor (3D+)**: 다차원 배열. (예: 컬러 이미지 `[H, W, C]`, 시계열 데이터 `[Batch, Time, Feature]`)

> **Key Point**: 신경망 연산에서 발생하는 에러의 90%는 이 '차원(Shape)'이 맞지 않아 발생합니다.

In [10]:
import numpy as np

# 텐서 차원 확인 예시
scalar = np.array(5)
vector = np.array([1, 2, 3])
matrix = np.array([[1, 2], [3, 4]])
tensor = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

print(f"Scalar Rank: {scalar.ndim}, Shape: {scalar.shape}")
print(f"Vector Rank: {vector.ndim}, Shape: {vector.shape}")
print(f"Matrix Rank: {matrix.ndim}, Shape: {matrix.shape}")
print(f"Tensor Rank: {tensor.ndim}, Shape: {tensor.shape}")

"""
[상세 주석]
- .ndim: 텐서의 차원(Rank)을 반환함.
- .shape: 각 차원의 크기를 튜플 형태로 반환함. AI 모델 설계 시 가장 빈번하게 체크해야 함.

- 시간 복잡도: O(1) (메타데이터 조회)
- 실행 환경: NumPy 1.2x 이상
"""

Scalar Rank: 0, Shape: ()
Vector Rank: 1, Shape: (3,)
Matrix Rank: 2, Shape: (2, 2)
Tensor Rank: 3, Shape: (2, 2, 2)


'\n[상세 주석]\n- .ndim: 텐서의 차원(Rank)을 반환함.\n- .shape: 각 차원의 크기를 튜플 형태로 반환함. AI 모델 설계 시 가장 빈번하게 체크해야 함.\n\n- 시간 복잡도: O(1) (메타데이터 조회)\n- 실행 환경: NumPy 1.2x 이상\n'

### 2. 행렬 곱셈(Dot Product)의 원리와 구현
행렬 곱셈은 앞 행렬의 **열(Column)** 개수와 뒤 행렬의 **행(Row)** 개수가 일치해야 연산이 가능합니다. 
이를 3중 `for`문으로 직접 구현해 보며 알고리즘의 복잡도를 이해합니다.

In [11]:
def manual_matrix_mul(A, B):
    """
    행렬 A와 B의 곱을 3중 for문으로 직접 구현합니다.
    A: (n x m) / B: (m x p) -> Result: (n x p)
    """
    n = len(A)
    m = len(A[0])
    p = len(B[0])
    
    # 0으로 채워진 결과 행렬 생성
    result = [[0 for _ in range(p)] for _ in range(n)]
    
    # 3중 Loop 연산
    for i in range(n):          # A의 행
        for j in range(p):      # B의 열
            for k in range(m):  # 공통 차원 (Dot Product 핵심)
                result[i][j] += A[i][k] * B[k][j]
                
    return result

# 테스트
A = [[1, 2], [3, 4]]
B = [[5, 6], [7, 8]]
print(f"Manual Result: {manual_matrix_mul(A, B)}")

"""
[상세 주석]
- i, j, k 세 루프가 중첩되어 데이터 크기 n에 대해 연산량이 n의 세제곱으로 증가함.
- 실무에서는 대규모 데이터 처리에 부적합하므로 벡터화(Vectorization)를 사용함.

- 시간 복잡도: O(n^3)
- 실행 환경: Pure Python
"""

Manual Result: [[19, 22], [43, 50]]


'\n[상세 주석]\n- i, j, k 세 루프가 중첩되어 데이터 크기 n에 대해 연산량이 n의 세제곱으로 증가함.\n- 실무에서는 대규모 데이터 처리에 부적합하므로 벡터화(Vectorization)를 사용함.\n\n- 시간 복잡도: O(n^3)\n- 실행 환경: Pure Python\n'

### 3. NumPy 벡터화(Vectorization) 성능 비교
NumPy는 내부적으로 C와 Fortran으로 구현되어 있어, 반복문 없이 배열 전체를 한 번에 연산합니다. 
이는 하드웨어 레벨의 병렬 연산(SIMD)을 활용하기 때문에 압도적으로 빠릅니다.

In [12]:
import time

# 400x400 대규모 행렬 생성
size = 400
A_np = np.random.rand(size, size)
B_np = np.random.rand(size, size)

# 1. Manual Loop 시간 측정
A_list, B_list = A_np.tolist(), B_np.tolist()
start = time.time()
manual_matrix_mul(A_list, B_list)
loop_time = time.time() - start

# 2. NumPy Vectorization 시간 측정
start = time.time()
np.dot(A_np, B_np)
vector_time = time.time() - start

print(f"Loop Time: {loop_time:.4f}s")
print(f"Vectorization Time: {vector_time:.4f}s")
print(f"Performance Gap: {loop_time / vector_time:.1f}x")

"""
[상세 주석]
- NumPy의 np.dot()은 BLAS(Basic Linear Algebra Subprograms) 라이브러리를 활용함.
- 데이터가 커질수록 Python의 동적 타이핑 오버헤드가 없는 벡터화 연산이 필수적임.
"""

Loop Time: 4.3490s
Vectorization Time: 0.0015s
Performance Gap: 2851.0x


'\n[상세 주석]\n- NumPy의 np.dot()은 BLAS(Basic Linear Algebra Subprograms) 라이브러리를 활용함.\n- 데이터가 커질수록 Python의 동적 타이핑 오버헤드가 없는 벡터화 연산이 필수적임.\n'

### 💡 디버깅 가이드: Shape Mismatch (차원 불일치)

> **[원인 분석]**
> 행렬 곱셈 시 `A.shape[1] != B.shape[0]`인 경우 발생합니다. (앞 행렬의 열 != 뒤 행렬의 행)
> 데이터 전처리 단계에서 행과 열을 혼동하거나, 전치(Transpose)를 누락했을 때 주로 나타납니다.

> **[재발 방지 팁]**
> 1. 연산 직전 반드시 `print(arr.shape)`를 습관화하세요.
> 2. `reshape(-1, 1)` 등을 활용해 명시적으로 차원을 고정하여 안전성을 확보하세요.

# Chapter 01-2. 브로드캐스팅(Broadcasting) & 메모리 연속성

이 섹션에서는 모양이 다른 배열 간의 연산을 가능하게 하는 **브로드캐스팅**의 수학적 원리와, 연산 속도에 직결되는 **메모리 레이아웃**의 물리적 구조를 **Deep Dive** 합니다.

### 1. 브로드캐스팅(Broadcasting)의 원리
브로드캐스팅은 NumPy가 연산 중에 작은 배열을 큰 배열의 모양에 맞춰 '가상으로' 확장하는 메커니즘입니다. 데이터를 실제로 복사하지 않으므로 메모리 점유율을 늘리지 않고도 고속 연산이 가능합니다.

**[브로드캐스팅 성립 조건]**
1. 두 배열의 차원 수가 다르면, 차원이 낮은 배열의 **왼쪽(앞쪽)**을 1로 채워 차원을 맞춤.
2. 각 차원에서 두 배열의 크기가 **동일**하거나, 어느 한 쪽의 크기가 **1**이어야 함.

In [13]:
import numpy as np

# 1. 브로드캐스팅 실습: (3, 3) 행렬 + (1, 3) 벡터
# matrix shape: (3, 3)
matrix = np.array([[10, 20, 30], 
                   [40, 50, 60], 
                   [70, 80, 90]])

# vector shape: (3,) -> 규칙 1에 의해 (1, 3)으로 간주됨
vector = np.array([1, 2, 3]) 

# 산술 연산 (Broadcasting 발생)
result = matrix + vector

print("Matrix (3x3):\n", matrix)
print("Vector (1x3 가상 확장):\n", vector)
print("Result:\n", result)

"""
[상세 주석]
- vector의 각 요소가 matrix의 각 행에 동일하게 더해짐.
- 실제 메모리 상에서 vector를 3행으로 복제하는 과정이 없으므로 매우 효율적임.

- 시간 복잡도: O(N*M) (N: 행, M: 열)
- 실행 환경: NumPy 기반 벡터 연산
"""

Matrix (3x3):
 [[10 20 30]
 [40 50 60]
 [70 80 90]]
Vector (1x3 가상 확장):
 [1 2 3]
Result:
 [[11 22 33]
 [41 52 63]
 [71 82 93]]


'\n[상세 주석]\n- vector의 각 요소가 matrix의 각 행에 동일하게 더해짐.\n- 실제 메모리 상에서 vector를 3행으로 복제하는 과정이 없으므로 매우 효율적임.\n\n- 시간 복잡도: O(N*M) (N: 행, M: 열)\n- 실행 환경: NumPy 기반 벡터 연산\n'

### 2. 메모리 연속성 (Memory Layout: C vs Fortran)
NumPy 배열은 메모리 상에 1차원적으로 펼쳐져 저장됩니다. 이 데이터를 읽는 순서(Layout)에 따라 CPU 캐시 효율이 결정되며, 이는 대규모 AI 연산 성능의 핵심 요소입니다.

- **C-style (Row-major)**: 행(Row)을 우선적으로 저장. (Python 기본값)
- **Fortran-style (Column-major)**: 열(Column)을 우선적으로 저장.

**[Deep Dive: 캐시 히트(Cache Hit)]**
데이터가 메모리에 물리적으로 붙어 있을수록 CPU는 다음 데이터를 미리 읽어올 수 있습니다. 행 중심 저장 방식(C-style)에서 행 방향 연산을 수행하면 속도가 극대화됩니다.

In [14]:
import time

# 10,000 x 10,000 대규모 행렬 생성
size = 10000
A_c = np.ones((size, size), order='C') # C-style (행 중심 저장)
A_f = np.ones((size, size), order='F') # F-style (열 중심 저장)

# Case 1: C-style에서 행 방향 합계 (효율적)
start = time.time()
np.sum(A_c, axis=1) 
c_time = time.time() - start

# Case 2: F-style에서 행 방향 합계 (비효율적 - 메모리 점프 발생)
start = time.time()
np.sum(A_f, axis=1) 
f_time = time.time() - start

print(f"C-style(Row-major) 연산 시간: {c_time:.4f}s")
print(f"F-style(Column-major) 연산 시간: {f_time:.4f}s")
print(f"성능 차이: {f_time / c_time:.2f}배")

"""
[상세 주석]
- axis=1(행 방향) 연산 시, C-style은 메모리를 순차적으로 읽지만 F-style은 큰 보폭으로 건너뛰며 읽어야 함.
- 딥러닝 모델의 텐서 구조 설계 시 연산 방향과 메모리 레이아웃을 일치시키는 것이 중요함.
"""

C-style(Row-major) 연산 시간: 0.1302s
F-style(Column-major) 연산 시간: 0.0389s
성능 차이: 0.30배


'\n[상세 주석]\n- axis=1(행 방향) 연산 시, C-style은 메모리를 순차적으로 읽지만 F-style은 큰 보폭으로 건너뛰며 읽어야 함.\n- 딥러닝 모델의 텐서 구조 설계 시 연산 방향과 메모리 레이아웃을 일치시키는 것이 중요함.\n'

### 💡 디버깅 가이드: 브로드캐스팅 Shape 에러

> **[원인 분석]**
> `ValueError: operands could not be broadcast together with shapes (4, 3) (4,)`
> 뒤쪽 차원(3 vs 4)이 일치하지 않고 어느 쪽도 1이 아닐 때 발생함.

> **[재발 방지 팁]**
> 1. `vector[:, np.newaxis]`를 사용하여 (4,)를 (4, 1)로 변환하면 (4, 3)과 연산 가능해짐.
> 2. `np.expand_dims()` 함수를 활용해 명시적으로 축을 추가하여 의도를 명확히 함.

### 1.4 축(Axis)의 확장과 슬라이싱 (Slicing)
데이터의 특정 부분을 추출하거나, AI 모델의 입력 규격에 맞게 차원을 맞추는 작업입니다.

- **슬라이싱(Slicing)**: 배열의 일부분을 선택함. (원본의 뷰(View)를 반환하여 메모리 효율적임)
- **차원 확장(np.newaxis)**: (N,) 벡터를 (N, 1) 또는 (1, N) 행렬로 변환하여 브로드캐스팅이나 모델 연산 준비를 함.

In [15]:
import numpy as np

# 3x3 행렬 생성
A = np.array([[1, 2, 3], 
              [4, 5, 6], 
              [7, 8, 9]])

# 1. 특정 열 추출 (Axis 1 방향 슬라이싱)
col_1 = A[:, 0]  # [1, 4, 7]

# 2. 차원 확장 (Newaxis 활용)
# (3,) -> (3, 1)로 변환 (모델 입력이나 브로드캐스팅 시 자주 사용)
col_expanded = col_1[:, np.newaxis]

print(f"Original Col Shape: {col_1.shape}")
print(f"Expanded Col Shape: {col_expanded.shape}")

"""
[상세 주석]
- A[:, 0]은 인덱싱 과정에서 차원이 하나 줄어듦(1D).
- np.newaxis를 추가하여 연산에 필요한 축을 명시적으로 생성함.

- 시간 복잡도: O(1) (데이터 복사 없음)
"""

Original Col Shape: (3,)
Expanded Col Shape: (3, 1)


'\n[상세 주석]\n- A[:, 0]은 인덱싱 과정에서 차원이 하나 줄어듦(1D).\n- np.newaxis를 추가하여 연산에 필요한 축을 명시적으로 생성함.\n\n- 시간 복잡도: O(1) (데이터 복사 없음)\n'

### 1.5 차원 조작 심화: Reshape & Squeeze
데이터의 총 개수는 유지하면서 형태(Shape)만 바꾸는 기법입니다.

- **Reshape**: 전체 요소 개수는 그대로 둔 채 차원의 모양을 변경함. `-1`을 쓰면 나머지 차원에 맞춰 자동 계산됨.
- **Squeeze**: 크기가 1인 불필요한 차원을 제거하여 연산을 단순화함.
- **Unsqueeze**: `np.expand_dims`와 유사하게 차원을 추가함.

In [16]:
# 12개의 요소를 가진 1차원 배열
data = np.arange(12) 

# (3, 4)로 변형 (3 * 4 = 12)
reshaped_data = data.reshape(3, -1) 

# 불필요한 차원 제거 예시 (1, 3, 4) -> (3, 4)
complex_tensor = reshaped_data.reshape(1, 3, 4)
squeezed_tensor = np.squeeze(complex_tensor)

print(f"Reshape (-1 사용): {reshaped_data.shape}")
print(f"Squeeze 결과: {squeezed_tensor.shape}")

"""
[상세 주석]
- reshape(3, -1)에서 -1은 12 / 3 = 4로 자동 계산됨.
- squeeze는 데이터의 본질은 건드리지 않고 '껍데기(차원)'만 정리하는 역할임.

- 시간 복잡도: O(1) (메모리 재배치 없이 메타데이터(Shape/Stride)만 변경)
"""

Reshape (-1 사용): (3, 4)
Squeeze 결과: (3, 4)


"\n[상세 주석]\n- reshape(3, -1)에서 -1은 12 / 3 = 4로 자동 계산됨.\n- squeeze는 데이터의 본질은 건드리지 않고 '껍데기(차원)'만 정리하는 역할임.\n\n- 시간 복잡도: O(1) (메모리 재배치 없이 메타데이터(Shape/Stride)만 변경)\n"

### 1.6 고유값과 고유벡터 (Eigenvalue & Eigenvector) 기초
- **개념**: 선형 변환(행렬 연산)을 가했을 때, 방향은 변하지 않고 **크기만 변하는 벡터**.
- **실무적 의미**: 데이터의 분산이 가장 큰 **'핵심 축(Direction)'**을 찾는 도구. 
- **활용**: 차원 축소 기술인 PCA(주성분 분석)의 핵심 수학 원리임.

In [17]:
# 선형대수 라이브러리(linalg) 활용
matrix = np.array([[1, 2], [2, 1]])

# 고유값(w)과 고유벡터(v) 계산
eigenvalues, eigenvectors = np.linalg.eig(matrix)

print(f"Eigenvalues: {eigenvalues}")
print(f"Eigenvectors:\n{eigenvectors}")

"""
[상세 주석]
- 고유벡터는 데이터의 정보량이 가장 많이 집중된 방향을 가리킴.
- 고유값은 그 방향으로의 정보량(중요도)을 의미함.

- 실행 환경: np.linalg 패키지 활용
"""

Eigenvalues: [ 3. -1.]
Eigenvectors:
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]


'\n[상세 주석]\n- 고유벡터는 데이터의 정보량이 가장 많이 집중된 방향을 가리킴.\n- 고유값은 그 방향으로의 정보량(중요도)을 의미함.\n\n- 실행 환경: np.linalg 패키지 활용\n'

### 💡 디버깅 가이드: Reshape 시 요소 개수 불일치 에러

> **[원인 분석]**
> `ValueError: cannot reshape array of size 12 into shape (3, 5)`
> 원본 데이터 요소 개수(12)와 목표하는 모양의 곱(3*5=15)이 일치하지 않을 때 발생함.

> **[재발 방지 팁]**
> 1. 직접 계산하기보다는 `-1` 인자를 활용하여 NumPy가 계산하게 하세요.
> 2. `arr.size`를 출력하여 전체 요소 개수를 먼저 확인하는 습관을 들입니다.